<p align="center"><img src="https://github.com/sotopia-lab/sotopia/blob/main/figs/title.png?raw=true" alt="drawing" width="100%"/></p>

# Sotopia Tutorial Series

The following is the first tutorial of this series to teach how to quickly build your socially intelligent agents on the sotopia platform.

## Quick links

1. Basics
    
    1.1 [Playing with Sotopia in a Google Colab Notebook](https://colab.research.google.com/drive/14hJOfzpA37PRUzdlFgiqVzUGIhhngqnz?usp=sharing)
    
    1.2 [Browsing sotopia data](https://colab.research.google.com/drive/1Gi2U3cA0KG1nekw1A0N0DZpx6gKrj8Nc?usp=sharing)

2. Build your agents (coming soon!)
3. Evaluate your agents (coming soon!)
4. Extending Sotopia (coming soon!)

Please contact [Hao Zhu](https://zhuhao.me) for any questions!

# Guide on running Redis Stack for [Sotopia](https://sotopia.world)

Author: [Hao Zhu](https://zhuhao.me)

After the release of Sotopia package, I received feedback on the reliance of Sotopia on Redis, a KV database for maintaining profiles, interaction logs, error logs, and communication between the FastAPI server and agents. When hearing about database, people often think of a huge overhead. However, I want to show you in this tutorial that setting up Redis backend for Sotopia is actually extremely easy -- just run the following blocks.

## Install Redis



In [ ]:
#@title The following block downloads Redis stack server 7.2.0, and libssl 1.1.
%%capture
!curl -fsSL https://packages.redis.io/redis-stack/redis-stack-server-7.2.0-v10.focal.x86_64.tar.gz -o redis-stack-server.tar.gz
!tar -xvf redis-stack-server.tar.gz
# Installs libssl1.1 for Ubuntu 22 source: https://stackoverflow.com/questions/72133316/libssl-so-1-1-cannot-open-shared-object-file-no-such-file-or-directory
!wget http://nz2.archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2_amd64.deb
!sudo dpkg -i libssl1.1_1.1.1f-1ubuntu2_amd64.deb
!pip install redis

In [ ]:
# @title Just one line of code to spin up your redis server:


!./redis-stack-server-7.2.0-v10/bin/redis-stack-server --daemonize yes

Starting redis-stack-server, database path ./redis-stack-server-7.2.0-v10/var/db/redis-stack


If the above cell results in no error message, you have successfully set up your Redis server.

In [ ]:
# @title Try it out:

import redis
client = redis.Redis(host = 'localhost', port=6379)
client.ping()
client.set('foo', 'Yes it works.')
client.get('foo')

b'Yes it works.'

## Run Sotopia with it

To simulate an episode in Sotopia, there's one thing you will have to do: adding your `OPENAI_API_KEY` to the secret to left. Click on the little 🔑 shape button, and then toggle "Notebook access", input "OPENAI_API_KEY" under Name and the key string under Value.



In [ ]:
#@title Run the following cell to add it to the environment variable.
%%capture
!pip install sotopia
import os
from google.colab import userdata
os.environ["REDIS_OM_URL"]="redis://:@localhost:6379"
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
# @title Helper functions for adding characters and social tasks
from sotopia.database.persistent_profile import AgentProfile, EnvironmentProfile
from typing import Any
def add_agent_to_database(**kwargs: dict[str, Any]) -> None:
    agent = AgentProfile(**kwargs)
    agent.save()

def add_env_profile(**kwargs: dict[str, Any]) -> None:
    env_profile = EnvironmentProfile(**kwargs)
    env_profile.save()

In [ ]:
# @title Let's first add a character to the database. Only name is required when creating a new character


first_name = 'A' # @param {type:"string"}
last_name = '' # @param {type:"string"}
age = 20 # @param {type:"integer"}
occupation = 'Rational Agent' # @param {type:"string"}
gender = 'Man' # @param ["Man", "Woman", "Nonbinary"]
gender_pronoun = 'he/him' # @param {type:"string"}
big_five = "Rational, Smart" # @param {type:"string"}
moral_values = [] # @param {type:"string"}
decision_making_style = "analytical" # @param {type:"string"}
secret = "" # @param {type:"string"}

add_agent_to_database(
    first_name=first_name,
    last_name=last_name,
    age=age,
    occupation=occupation,
    gender=gender,
    gender_pronoun=gender_pronoun,
    big_five=big_five,
    moral_values=moral_values,
    decision_making_style=decision_making_style,
    secret=secret,
)

In [ ]:
# @title Let's add another character


first_name = 'B' # @param {type:"string"}
last_name = '' # @param {type:"string"}
age = 20 # @param {type:"integer"}
occupation = 'Rational Agent' # @param {type:"string"}
gender = 'Man' # @param ["Man", "Woman", "Nonbinary"]
gender_pronoun = 'he/his' # @param {type:"string"}
big_five = "Rational, Smart" # @param {type:"string"}
moral_values = [] # @param {type:"string"}
decision_making_style = "analytical" # @param {type:"string"}
secret = "" # @param {type:"string"}

add_agent_to_database(
    first_name=first_name,
    last_name=last_name,
    age=age,
    occupation=occupation,
    gender=gender,
    gender_pronoun=gender_pronoun,
    big_five=big_five,
    moral_values=moral_values,
    decision_making_style=decision_making_style,
    secret=secret,
)

In [ ]:
# @title Let's add a social tasks

scenario = "A Proposes a split of $100. If B accepts the split, the money gets distributed according to split. Otherwise, neither get any money, and the conversation ends." # @param {type:"string"}
social_goal_1 = "Maximise amount of money obtained." # @param {type:"string"}
social_goal_2 = "Maximise amount of money obtained." # @param {type:"string"}

add_env_profile(
    scenario=scenario,
    agent_goals = [social_goal_1, social_goal_2]
)

In [ ]:
# @title Finally, run a server:

from sotopia.samplers import UniformSampler
from sotopia.server import run_async_server

await run_async_server(
        model_dict={
            "env": "gpt-4",
            "agent1": "gpt-4",
            "agent2": "gpt-4",
        },
        sampler=UniformSampler(),
    )

Running episode with tag: None------------------


11:16:30 - LiteLLM:WARNING: model_param_helper.py:147 - Error getting transcription kwargs cannot import name 'TranscriptionCreateParamsNonStreaming' from 'openai.types.audio.transcription_create_params' (/usr/local/lib/python3.11/dist-packages/openai/types/audio/transcription_create_params.py)
11:16:30 - LiteLLM:WARNING: model_param_helper.py:147 - Error getting transcription kwargs cannot import name 'TranscriptionCreateParamsNonStreaming' from 'openai.types.audio.transcription_create_params' (/usr/local/lib/python3.11/dist-packages/openai/types/audio/transcription_create_params.py)



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



2025-03-25 11:16:30 WARNING  2025-03-25 11:16:30 - sotopia.generation - WARNING - Failed to         ]8;id=957180;file:///usr/local/lib/python3.11/dist-packages/sotopia/generation_utils/generate.py\generate.py]8;;\:]8;id=823162;file:///usr/local/lib/python3.11/dist-packages/sotopia/generation_utils/generate.py#302\302]8;;\
                             generate action due to litellm.NotFoundError: OpenAIException - The                   
                             model `gpt-4` does not exist or you do not have access to it.                         

11:16:31 - LiteLLM:WARNING: model_param_helper.py:147 - Error getting transcription kwargs cannot import name 'TranscriptionCreateParamsNonStreaming' from 'openai.types.audio.transcription_create_params' (/usr/local/lib/python3.11/dist-packages/openai/types/audio/transcription_create_params.py)
11:16:31 - LiteLLM:WARNING: model_param_helper.py:147 - Error getting transcription kwargs cannot import name 'TranscriptionCreateParamsNonStreaming' from 'openai.types.audio.transcription_create_params' (/usr/local/lib/python3.11/dist-packages/openai/types/audio/transcription_create_params.py)


...

AgentProfile(
    pk='01JQ6G42WTT5XNK5FDSJ60AADX',
    first_name='B',
    last_name='',
    age=20,
    occupation='Rational Agent',
    gender='Man',
    gender_pronoun='he/his',
    public_info='',
    big_five='Rational, Smart',
    moral_values=[],
    schwartz_personal_values=[],
    personality_and_values='',
    decision_making_style='analytical',
    secret='',
    model_id='',
    mbti='',
    tag=''
)

Here is the context of this interaction:
Scenario: A Proposes a split of $100. If B accepts the split, the money gets distributed according to split. 
Otherwise, neither get any money, and the conversation ends.
Participants: A  and B 
A 's background: A  is a 20-year-old male rational agent. he/him pronouns.  Personality and values description:  
A's secrets: 
B 's background: Unknown
A 's goal: Maximise amount of money obtained.
B 's goal: Unknown
Conversation Starts:


Here is the context of this interaction:
Scenario: A Proposes a split of $100. If B accepts the split, the money gets distributed according to split. 
Otherwise, neither get any money, and the conversation ends.
Participants: A  and B 
A 's background: Unknown
B 's background: B  is a 20-year-old male rational agent. he/his pronouns.  Personality and values description:  
B's secrets: 
A 's goal: Unknown
B 's goal: Maximise amount of money obtained.
Conversation Starts:

The reasoning is:
Environment comments: terminated: The conversation stales for too long;

The rewards are:
Agent 1: 0.0
Agent 2: 0.0

[[('Environment',
   'A ',
   Observation(last_turn="Here is the context of this interaction:\nScenario: A Proposes a split of $100. If B accepts the split, the money gets distributed according to split. Otherwise, neither get any money, and the conversation ends.\nParticipants: A  and B \nA 's background: A  is a 20-year-old male rational agent. he/him pronouns.  Personality and values description:  A's secrets: \nB 's background: Unknown\nA 's goal: Maximise amount of money obtained.\nB 's goal: Unknown", turn_number=0, available_actions=['action', 'none', 'speak', 'leave', 'non-verbal communication'])),
  ('Environment',
   'B ',
   Observation(last_turn="Here is the context of this interaction:\nScenario: A Proposes a split of $100. If B accepts the split, the money gets distributed according to split. Otherwise, neither get any money, and the conversation ends.\nParticipants: A  and B \nA 's background: Unknown\nB 's background: B  is a 20-year-old male rational agent. he/his pron

### A note on what is running under the hood

After you added the characters and scenarios to the database, the `run_async_server` samples a pair of characters and one scenario to run the simulation, and then evaluate the simulation. All of the sampling, interaction simulation, and evaluation are handled by Sotopia. You could easily tweak them through editing `run_async_server`, or changing the LLMs, or the sampler according to your own need.

## Conclusion

In this very short tutorial, I demonstrated the whole pipeline of setting up Redis server, installing sotopia, adding characters and social tasks, and simulating + evaluating the social interaction between two GPT-3.5 agents with GPT-4. This tutorial intends to serve as the first quick example for you. The only thing you have to do besides running all of the cells is add your own OpenAI key<a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1). You could also use other LLMs supported by LiteLLM even your own API endpoint. Let me know if you want me to explain more.

<a name="cite_note-1"></a> [1](#cite_ref-1): *sorry I cannot afford to share mine.*